# Demo: LLM-Assisted Coding for a Factorial Function

Welcome! This notebook is a short demonstration of how you can collaborate with an LLM to generate a Python function and its corresponding tests for the `factorial` function.

In [1]:
import sys
is_colab = 'google.colab' in sys.modules
is_colab

True

In [2]:
wd = '../../../../../'
if is_colab:
    wd = '/content/drive/Othercomputers/Mac Studio/'
wd

'/content/drive/Othercomputers/Mac Studio/'

In [3]:
if is_colab:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%pip install -qqq -r "{wd}materials/v2/C1 - Fundamentals/requirements3-5.txt"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 20.2 MB/s eta 0:00:00


In [5]:
import litellm
import os

if is_colab:
    from google.colab import userdata
    litellm.openai_key = userdata.get("OPENAI_API_KEY")
else:
    from dotenv import load_dotenv
    load_dotenv()
    litellm.openai_key = os.getenv("OPENAI_API_KEY")

# If using Vocareum, you can also set your API key here directly
# Uncomment and replace the string with your actual Vocareum API key
# litellm.openai_key = "voc-**********"

if (litellm.openai_key or "").startswith("voc-"):
    litellm.api_base = "https://openai.vocareum.com/v1"
    print("Detected vocareum API key. Using Vocareum OpenAI API base URL.")

## 1. Prompting for a Factorial Function

First, we'll ask the LLM to write a Python function that calculates the factorial of a number. We'll specify that it should be a recursive function named `factorial`.

In [6]:
SYSTEM_PROMPT = """You are a helpful coding assistant. Return only the Python code, with no explanation or preamble."""
USER_PROMPT = """Write a Python function that implements the factorial algorithm using recursion.
The name of the function should be `factorial`.
The function should take a single non-negative integer as input.
Include a docstring explaining what the function does."""

Next, we send our prompt to the LLM.

In [7]:
from litellm import completion

response = completion(
    model="gpt-5-mini",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print("Generated Code:\n")
print(response.choices[0].message.content)

Generated Code:

def factorial(n):
    """Compute and return the factorial of a non-negative integer n using recursion.

    Parameters:
        n (int): A non-negative integer whose factorial is to be computed.

    Returns:
        int: The factorial of n (n!).

    Raises:
        TypeError: If n is not an integer.
        ValueError: If n is negative.
    """
    if not isinstance(n, int):
        raise TypeError("factorial() only accepts integers")
    if n < 0:
        raise ValueError("factorial() not defined for negative values")
    if n == 0 or n == 1:
        return 1
    return n * factorial(n - 1)


Now, we'll take the code generated by the LLM and place it into a cell to make it usable.

In [ ]:
def factorial(n):
    """
    Compute the factorial of a non-negative integer n using recursion.

    Args:
        n (int): A non-negative integer whose factorial is to be computed.

    Returns:
        int: The factorial of n (n!).

    Raises:
        TypeError: If n is not an integer.
        ValueError: If n is negative.
    """
    if not isinstance(n, int):
        raise TypeError("factorial() requires an integer input")
    if n < 0:
        raise ValueError("factorial() not defined for negative values")
    if n <= 1:
        return 1
    return n * factorial(n - 1)

## 2. Quick Functional Check

Let's do a quick check to see if the function works as expected for a simple case, like calculating the factorial of 5 (which is $5! = 120$).

In [ ]:
input_val = 5
expected_val = 120

output_val = factorial(input_val)

print(f"Input: {input_val}")
print(f"Output: {output_val}")
print(f"Expected: {expected_val}")
print(f"Match: {output_val == expected_val}")
print()

if output_val == expected_val:
    print("Test case passed! ✅")
else:
    print("Test case failed. ❌")

Input: 5
Output: 120
Expected: 120
Match: True

Test case passed! ✅


## 3. Prompting for Test Cases

The function seems to work for a basic case. Now, let's ask the LLM to generate a comprehensive test suite using Python's `unittest` framework. We'll ask for tests that cover:

* A basic positive integer.
* An edge case (input `0`).
* An invalid input (a negative number).

In [ ]:
SYSTEM_PROMPT = """You are a helpful coding assistant that writes unit tests using the unittest framework. Return only the code, no explanations."""
USER_PROMPT = """Please generate test cases for the `factorial` function.
The test cases should be in a class called `TestFactorial` that inherits from `unittest.TestCase`.
Include tests for a positive integer, the number zero, and a negative number.
The test for the negative number should check that a `ValueError` is raised.
Do not include any imports or a main block.
"""

In [ ]:
response = completion(
    model="gpt-5-mini",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print("Generated Test Cases:\n")
print(response.choices[0].message.content)

Generated Test Cases:

class TestFactorial(unittest.TestCase):
    def test_positive_integer(self):
        self.assertEqual(factorial(5), 120)
        self.assertEqual(factorial(1), 1)

    def test_zero(self):
        self.assertEqual(factorial(0), 1)

    def test_negative_integer_raises(self):
        with self.assertRaises(ValueError):
            factorial(-3)


## 4. Running the Tests

Finally, let's paste the LLM-generated test code into a cell and run it. Let's see what happens!

In [ ]:
import unittest


class TestFactorial(unittest.TestCase):
    def test_positive_integer(self):
        self.assertEqual(factorial(5), 120)
        self.assertEqual(factorial(1), 1)

    def test_zero(self):
        self.assertEqual(factorial(0), 1)

    def test_negative_integer_raises(self):
        with self.assertRaises(ValueError):
            factorial(-3)

In [ ]:
# Run the tests
unittest.main(argv=["notebook-tests"], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.002s

OK


### Reflection

All the tests passed!

This won't always be the case, and you may need to modify the prompts for both the code and the tests depending on your specific requirements.

For instance, do we want the doc strings to be formatted in a certain way? Or do we have custom Exceptions we want to raise for invalid inputs? Do we have other edge cases we want to account for, such as non-integer inputs?

These are all things you can specify in your prompts to the LLM.